In [13]:
# No Notebook/Colab, instale a biblioteca primeiro
# !pip install datasets pandas pyarrow

from datasets import load_dataset
import pandas as pd

print("Carregando datasets da Amazon (isso pode levar um momento)...")

# 1. Carrega o subconjunto de Português (pt) e Espanhol (es)
#ds_pt = load_dataset("mteb/amazon_reviews_multi", "pt", split="train")
ds_pt = load_dataset("ruanchaves/b2w-reviews01", split="train")
ds_es = load_dataset("mteb/amazon_reviews_multi", "es", split="train")

# 2. Converte para Pandas DataFrame
df_pt = ds_pt.to_pandas()
df = ds_es.to_pandas()

# 3. Unifica e seleciona as colunas necessárias
#df = pd.concat([df_pt, df_es], ignore_index=True)
#df = df[['review_body', 'stars', 'language']]

print(df_pt.head())
print(df.tail())

print(f"Sucesso! Dataset unificado com {len(df)} e {len(df_pt)} registros.")
#print(df['language'].value_counts())

Carregando datasets da Amazon (isso pode levar um momento)...


Generating train split: 0 examples [00:00, ? examples/s]C:\Users\Bruno\Desktop\JupyterProject\.venv1\Lib\site-packages\datasets\download\streaming_download_manager.py:765: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)
Generating train split: 132373 examples [00:13, 9844.89 examples/s] 


       submission_date                                        reviewer_id  \
0  2018-01-01 00:11:28  d0fb1ca69422530334178f5c8624aa7a99da47907c44de...   
1  2018-01-01 00:13:48  014d6dc5a10aed1ff1e6f349fb2b059a2d3de511c7538a...   
2  2018-01-01 00:26:02  44f2c8edd93471926fff601274b8b2b5c4824e386ae4f2...   
3  2018-01-01 00:35:54  ce741665c1764ab2d77539e18d0e4f66dde6213c9f0863...   
4  2018-01-01 01:00:28  7d7b6b18dda804a897359276cef0ca252f9932bf4b5c8e...   

  product_id                                       product_name  \
0  132532965  Notebook Asus Vivobook Max X541NA-GO472T Intel...   
1   22562178               Copo Acrílico Com Canudo 500ml Rocie   
2  113022329  Panela de Pressão Elétrica Philips Walita Dail...   
3  113851581               Betoneira Columbus - Roma Brinquedos   
4  131788803  Smart TV LED 43" LG 43UJ6525 Ultra HD 4K com C...   

    product_brand      site_category_lv1       site_category_lv2  \
0            None            Informática                Notebook  

2. Informações Técnicas e Tamanho
Aqui estão os dados atualizados para o seu planejamento de infraestrutura:

Tamanho Físico (Disco): Aproximadamente 160 MB para os arquivos compactados (PT + ES).

Tamanho em Memória (RAM): Após carregar no Pandas, espere algo em torno de 450 MB a 600 MB. É um tamanho excelente para os limites gratuitos do Google Colab ou máquinas locais.

Qualidade: São 400.000 avaliações reais (200k de cada país), todas rotuladas de 1 a 5 estrelas.
<br><br>

---



3. Ajuste no Pré-processamento (JSON-ready)
Para garantir que o seu modelo aprenda bem os dois idiomas e esteja pronto para o formato JSON da sua API, use este bloco para finalizar a limpeza:
<br>


---



In [ ]:
# Transformando em binário (Positivo/Negativo)
# Removemos as 3 estrelas (neutro) para simplificar a classificação no Hackathon
df = df[df['stars'] != 3].copy()
df['label'] = df['stars'].apply(lambda x: 1 if x > 3 else 0)

# Limpeza rápida
df['review_body'] = df['review_body'].str.lower().str.replace('[^\w\s]', '', regex=True)

print("Exemplo de dado processado:")
print(df[['review_body', 'label', 'language']].head())

Código: Treino e Comparação de Performance (PT vs ES)
<br><br>


---



In [ ]:
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score

# 1. Carregar Dados de forma estável via Hugging Face
print("A carregar dados...")
ds_pt = load_dataset("amazon_reviews_multi", "pt", split="train")
ds_es = load_dataset("amazon_reviews_multi", "es", split="train")

df = pd.concat([ds_pt.to_pandas(), ds_es.to_pandas()], ignore_index=True)

# 2. Pré-processamento
df = df[df['stars'] != 3].copy()
df['label'] = df['stars'].apply(lambda x: 1 if x > 3 else 0)

# 3. Divisão Treino/Teste mantendo a referência do idioma
X = df[['review_body', 'language']]
y = df['label']

X_train_df, X_test_df, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Treinar o Pipeline
# Nota: Passamos apenas a coluna de texto para o fit
model_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=10000, ngram_range=(1, 2))),
    ('clf', LogisticRegression(max_iter=1000))
])

print("A treinar o modelo bilíngue...")
model_pipeline.fit(X_train_df['review_body'], y_train)

# 5. Avaliação Comparativa por Idioma
metrics = []

for lang in ['pt', 'es']:
    # Filtrar o set de teste por idioma
    mask = X_test_df['language'] == lang
    X_lang = X_test_df[mask]['review_body']
    y_lang = y_test[mask]

    # Predição
    preds = model_pipeline.predict(X_lang)

    # Cálculo de métricas
    acc = accuracy_score(y_lang, preds)
    f1 = f1_score(y_lang, preds)

    metrics.append({'Idioma': 'Português' if lang == 'pt' else 'Espanhol',
                    'Acurácia': acc, 'F1-Score': f1})

results_df = pd.DataFrame(metrics)

# 6. Visualização para o Pitch
plt.figure(figsize=(10, 6))
df_plot = results_df.melt(id_vars='Idioma', var_name='Métrica', value_name='Valor')
sns.barplot(data=df_plot, x='Idioma', y='Valor', hue='Métrica', palette='viridis')

plt.title('Performance do Modelo por Idioma', fontsize=15)
plt.ylim(0.8, 1.0) # Zoom para ver as diferenças
plt.ylabel('Score (0 a 1)')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

print("\nTabela de Resultados:")
print(results_df)

# Guardar o modelo final
joblib.dump(model_pipeline, 'modelo_multilingue_final.pkl')

Por que isto é importante para o seu Hackathon?
Validação de Viés: Se a acurácia no Espanhol for muito menor que no Português, o júri pode questionar a qualidade dos dados. Este gráfico mostra que o modelo é robusto em ambos.

Visualização Profissional: Slides com gráficos de barras bem acabados (usando o seaborn) passam muito mais credibilidade do que apenas números num terminal.

F1-Score: Incluí o F1-Score porque ele é mais "honesto" que a Acurácia se houver um desequilíbrio entre comentários positivos e negativos.
<br><br>


---



Com a sua base de dados unificada (Português e Espanhol) e o modelo treinado, o passo final é a API. Ela será o "coração" do seu projeto no Hackathon, permitindo que qualquer aplicação consuma a inteligência que você criou.

Para isso, utilizaremos o FastAPI pela sua velocidade e a biblioteca langdetect para identificar o idioma automaticamente.

1. Requisitos do Sistema
No ambiente onde a API vai rodar (seu computador ou servidor), instale estas bibliotecas:
<br>



```
# pip install fastapi uvicorn joblib scikit-learn langdetect
```

<br>
2. O Código da API (main.py)
Este script carrega o arquivo modelo_multilingue_final.pkl que geramos no passo anterior e expõe um endpoint de predição.

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import joblib
from langdetect import detect, DetectorFactory

# Garante que os resultados da detecção de idioma sejam determinísticos
DetectorFactory.seed = 0

# 1. Definição do App
app = FastAPI(
    title="API de Sentimento Multilíngue (PT/ES)",
    description="API para classificação de sentimento e detecção de idioma para o Hackathon.",
    version="1.0.0"
)

# 2. Carregar o Modelo (Pipeline com TF-IDF + Logistic Regression)
try:
    model = joblib.load('modelo_multilingue_final.pkl')
    print("Modelo carregado com sucesso!")
except FileNotFoundError:
    print("ERRO: O arquivo 'modelo_multilingue_final.pkl' não foi encontrado.")

# 3. Modelos de Dados (JSON Schema)
class SentimentRequest(BaseModel):
    text: str

class SentimentResponse(BaseModel):
    text: str
    sentiment: str
    probability: float
    language: str
    status: str

# 4. Rotas
@app.get("/")
def home():
    return {"message": "API Online. Vá para /docs para testar via Swagger."}

@app.post("/predict", response_model=SentimentResponse)
async def predict_sentiment(request: SentimentRequest):
    if not request.text.strip():
        raise HTTPException(status_code=400, detail="O texto fornecido está vazio.")

    try:
        # A. Detecção de Idioma
        # Identifica se é 'pt', 'es', etc.
        lang = detect(request.text)

        # B. Predição de Sentimento
        # O modelo já processa o texto bruto via TF-IDF (Pipeline)
        prediction = model.predict([request.text])[0]
        probabilities = model.predict_proba([request.text])[0]

        # Mapeamento: 1 -> Positivo, 0 -> Negativo
        sentiment_label = "Positivo" if prediction == 1 else "Negativo"
        confidence = float(probabilities.max())

        return {
            "text": request.text,
            "sentiment": sentiment_label,
            "probability": round(confidence, 4),
            "language": lang,
            "status": "success"
        }

    except Exception as e:
        return {
            "text": request.text,
            "sentiment": "N/A",
            "probability": 0.0,
            "language": "unknown",
            "status": f"Error: {str(e)}"
        }

if __name__ == "__main__":
    import uvicorn
    # Inicia o servidor na porta 8000
    uvicorn.run(app, host="0.0.0.0", port=8000)

3. Por que esta API é excelente para o seu projeto?
Detecção Automática: Ao retornar o campo "language": "es", você mostra que sua solução é inteligente. Se o usuário mandar "Me encanta este produto", a API dirá que o idioma é Espanhol e o sentimento é Positivo.

Validação de Dados: O uso do BaseModel (Pydantic) garante que, se alguém tentar enviar algo que não seja texto, a API retornará um erro 422 Unprocessable Entity automaticamente, protegendo seu código.

Documentação Viva (Swagger): Ao rodar a API e acessar http://localhost:8000/docs, você terá uma interface visual pronta. Dica de Hackathon: Use essa tela durante a apresentação para mostrar o modelo funcionando em tempo real para os jurados.

Como testar no terminal (Curl)
Enquanto a API estiver rodando, você ou seu time de Front-end podem testar assim:
<br><br>


---



In [ ]:
curl -X 'POST' \
  'http://localhost:8000/predict' \
  -H 'Content-Type: application/json' \
  -d '{ "text": "Este serviço é horrível e não funciona." }'

Resultado esperado (JSON):
<br><br>


---



In [ ]:
{
  "text": "Este serviço é horrível e não funciona.",
  "sentiment": "Negativo",
  "probability": 0.9854,
  "language": "pt",
  "status": "success"
}